#  Naive Bayes klasifikator

**"A probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features"**

# Pregled
* Generativni modeli
* Naive Bayes klasifikator
* Primer rada sa tekstom: twitter sentiment analysis na tvitovima


# Generativni modeli
* Klasifikatori se uobičajeno dele na dve glavne grupe: **diskriminativni** i **generativni**
* **Diskriminativni modeli** modeluju uslovnu verovatnoću $P(C|X=x)$, tj. verovatnoću da neki ulazni feature ili feature vektor pripada određenoj klasi
  * Predikcije iz ovog modela izvlačimo direktno
  * Primeri: logistička regresija, k-NN, SVM, neuralne mreže
* **Generativni modeli** modeluju združenu distribuciju $P(X, C)$
  * Predikcije iz ovog modela izvlačimo kao $P(C|X) = P(X, C) / P(X)$, ali kako je za dato $X$ verovatnoća $P(X)$ fiksna, važi $P(C|X) \sim P(X, C)$
  * Primeri: HMM, Naive Bayes, VAE, GAN

# Naive Bayes klasifikator
* Klasifikator koji koristi Bajesovu formulu
* Bajesova formula: $P(C|X) = \frac{P(X|C)P(C)}{P(X)} = \frac{P(X|C)P(C)}{\sum{P(X|C)P(C)}}$
  * $P(C)$ = *prior*
  * $P(X|C)$ = *likelihood*, $P(X)$ = *evidence*
  * $P(C|X)$ = *posterior*
* Kao što je već pomenuto $P(C|X) ~ P(X|C)P(C)$
  * Dakle: "verovatnoća da feature vektor X pripada klasi C je proporcionalna verovatnoći da bi neki primerak klase C imao feature vektor X, skalirano sa ukupnom verovatnoćom pojavljivanja klase C"
* $P(C)$ lako računamo kao broj pojavljivanja klase C u trening skupu, podeljeno sa veličinom trening skupa
  * $P(Klasa) = \frac{|Klasa~u~trening~skupu|}{|Trening~skup|}$
* Kako izračunati $P(X|C)$? 
  * Naive Bayes pretpostavlja *nezavisnost ulaznih promenljivih*, pa je $P(X|C) = P(x_1|C) * P(x_2|C) * \dots * P(x_N|C)$
* Ali kako izračunati $P(x_1|C)$, tj. verovatnoću da bi neki primerak klase C imao imao prvi feature jednak $x_1$?
  * U obradi teksta se često koristi *Multinomial Naive Bayes* sa BoW reprezentacijom koja čuva brojeve pojavljivanja
  * Podsetnik: u BoW jedan feature je jedna reč tj. broj pojavljivanja te reči u tekstu
  * Sada je dakle $P(x_1|C)$ verovatnoća da neka klasa dokumenata sadrži reč $x_1$, što lako računamo kao frekvenciju ove reči u tekstovima za koje znamo da su iz klase $C$
  * $P(Reč_i|Klasa) = \frac{broj(Reč_i, Klasa)}{broj(*, Klasa)}$
  * **Nazad na početak**: $P(Klasa|Skup Reči) \sim P(Klasa) * P(Reč_1 | Klasa) * P(Reč_2 | Klasa) * ... * P(Reč_N | Klasa)$
  * **Predstavljeno preko BoW**: $P(Klasa|BoW vektor) \sim P(Klasa) * \prod{P(Reč_i|Klasa)^{BoW[Reč_i]}}$
  * Očigledno, biramo klasu za koju je ova vrednost najveća
* **Problem 1**: kada je $N$ veliko množimo/stepenujemo mnogo brojeva od 0 do 1 što loše utiče na preciznost
  * Rešenje: logaritmujemo sve! Sada je $P(Klasa|BoW vektor) \sim \log(P(Klasa)) + \sum{BoW[Reč_i]*\log(P(Reč_i|Klasa))}$
* **Problem 2**:  Šta ako je $broj(Klasa)=0$? Deljenje nulom?
  * Rešenje: **Laplace Smoothing**
  * Uvodimo konstantu $\alpha$ (uglavnom jednaku 1) i smatramo da svaka klasa sadrži barem toliko od svake reči
  * Sada imamo za $\alpha=1$: $P(Reč_i|Klasa) = \frac{broj(Reč_i, Klasa) + 1}{broj(*, Klasa) + |Vocab|}$
* [Više o Naive Bayes](http://cs229.stanford.edu/notes/cs229-notes2.pdf)

## Naive Bayes - konačne formule
* $P(Klasa|BoW vektor) \sim \log(P(Klasa)) + \sum{BoW[Reč_i]*\log(P(Reč_i|Klasa))}$




---



* $P(Klasa) = \frac{|Klasa~u~trening~skupu|}{|Trening~skup|}$



---




* $P(Reč_i|Klasa) = \frac{broj(Reč_i, Klasa) + \alpha}{broj(*, Klasa) + |Vocab|\alpha}$

In [0]:
# TODO

# Primer rada sa tekstom: twitter sentiment analysis
*  Problem: klasifikovati tvitove na osnovu "sentimenta": (neutralan, pozitivan, negativan)
*  [Kaggle dataset](https://www.kaggle.com/c/twitter-sentiment-analysis2)
*  Potreban ceo proces čišćenja i obrade podataka
*  Potrebna feature-izacija (npr. BoW)
*  Jedan od mogućih modela može biti Multinomial Naive Bayes
*  Potrebna podela skupa podataka na trening/validacioni/test skup radi evaluacije rešenja
*  [Primer rešenja](https://github.com/rand0musername/twitter-sentiment)

